In [1]:
import pickle
import math
import json

In [2]:
with open('financial_statements.pickle', 'rb') as fp:
    data = pickle.load(fp)

In [3]:
def extract_lines_to_array(fp):
    return [*map(lambda line: line.strip(), fp.readlines())]

with open('dates.txt', 'r') as fp:
    dates = extract_lines_to_array(fp)

with open('bankrupt_companies_full.txt', 'r') as fp:
    bankrupt_companies = set(extract_lines_to_array(fp))

In [4]:
[x for x in data[0]]

['Revenues',
 'Total Equity',
 'Treasury Stock',
 'Share Capital',
 'Dividends',
 'Retained Earnings',
 'Net Change in Cash',
 'Total Noncurrent Liabilities',
 'Long Term Debt',
 'Net Profit',
 'Current Assets',
 'Preferred Equity',
 'financial year end (month)',
 'Total Assets',
 'ticker',
 'Receivables',
 'SimFin ID',
 'Minorities',
 'Short term debt',
 'Total Noncurrent Assets',
 'industry code',
 'company',
 'Goodwill',
 'Cash and Cash Equivalents',
 'Share Price',
 'EBIT',
 'Net PP&E',
 'Total Liabilities',
 'Accounts Payable',
 'EBITDA',
 'Common Shares Outstanding',
 'COGS',
 'Current Liabilities',
 'Avg. Basic Shares Outstanding',
 'Intangible Assets',
 'Cash From Operating Activities',
 'Net Income from Discontinued Op.',
 'Cash From Financing Activities',
 'SG&A',
 'Equity Before Minorities',
 'Avg. Diluted Shares Outstanding',
 'Cash From Investing Activities']

In [5]:
def cast_to_float_and_adjust(item, column):
    try: 
        return float(item) if column == 'Share Price' else float(item) * 1000.0 if 'Shares' in column else float(item) * 1000000.0
    except ValueError:
        return None
    
def fill_blanks(obj, column):
    most_recent_value = None
    for i in range(len(obj[column])):
        if obj[column][i]:
            most_recent_value = obj[column][i]
        else:
            obj[column][i] = most_recent_value

for obj in data:
    for key in obj:
        if isinstance(obj[key], list):
            obj[key] = [*map(lambda item: cast_to_float_and_adjust(item, key), obj[key])]
    fill_blanks(obj, 'Common Shares Outstanding')

In [6]:
# X21	sales (n) / sales (n-1) 
# X22	profit on operating activities / total assets 
# X23	net profit / sales 
# X24	gross profit (in 3 years) / total assets 
# X25	(equity - share capital) / total assets 
# X26	(net profit + depreciation) / total liabilities 
# X27	profit on operating activities / financial expenses 
# X28	working capital / fixed assets 
# X29	logarithm of total assets 
# X30	(total liabilities - cash) / sales 
# X31	(gross profit + interest) / sales 
# X32	(current liabilities * 365) / cost of products sold 
# X33	operating expenses / short-term liabilities 
# X34	operating expenses / total liabilities 
# X35	profit on sales / total assets 
# X36	total sales / total assets 
# X37	(current assets - inventories) / long-term liabilities 
# X38	constant capital / total assets 
# X39	profit on sales / sales 
# X40	(current assets - inventory - receivables) / short-term liabilities 

# X41	total liabilities / ((profit on operating activities + depreciation) * (12/365)) 
# X42	profit on operating activities / sales 
# X43	rotation receivables + inventory turnover in days 
# X44	(receivables * 365) / sales 
# X45	net profit / inventory 
# X46	(current assets - inventory) / short-term liabilities 
# X47	(inventory * 365) / cost of products sold 
# X48	EBITDA (profit on operating activities - depreciation) / total assets 
# X49	EBITDA (profit on operating activities - depreciation) / sales 
# X50	current assets / total liabilities 
# X51	short-term liabilities / total assets 
# X52	(short-term liabilities * 365) / cost of products sold) 
# X53	equity / fixed assets 
# X54	constant capital / fixed assets 
# X55	working capital 
# X56	(sales - cost of products sold) / sales 
# X57	(current assets - inventory - short-term liabilities) / (sales - gross profit - depreciation) 
# X58	total costs /total sales 
# X59	long-term liabilities / equity 
# X60	sales / inventory 
# X61	sales / receivables 
# X62	(short-term liabilities *365) / sales 
# X63	sales / short-term liabilities 
# X64	sales / fixed assets

def get_features(obj):
    def catchError(obj, i, func):
        try:
            return func(obj, i)
        except (TypeError, ZeroDivisionError, ValueError):
            return '';
        
    def get_first_item_of_year(data):
        seen_year = set()
        res = []
        for item in data:
            year = item['date'][-2:]
            if year not in seen_year:
                seen_year.add(year)
                res.append(item)
        return res

    funcs = {
        # X1:net profit / total assets 
        'x1': lambda obj, i: obj['Net Profit'][i] / obj['Total Assets'][i],
        
        # X2:total liabilities / total assets 
        'x2': lambda obj, i: obj['Total Liabilities'][i] / obj['Total Assets'][i],
        
        # X3:working capital / total assets 
        # working capital = current assets - current liabilities 
        'x3': lambda obj, i: (obj['Current Assets'][i] - obj['Current Liabilities'][i]) / obj['Total Assets'][i], 
        
        # X4:current assets / short-term liabilities 
        # assume short term liability = current liability
        'x4': lambda obj, i:  obj['Current Assets'][i] / obj['Current Liabilities'][i], 
        
        # X5:[(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365
        # cash + short-term securities + receivables = current assets
        #  Operating expenses - depreciation = sg&A   
        'x5': lambda obj, i: (obj['Current Assets'][i]-obj['Current Liabilities'][i]) / obj['SG&A'][i] * 365,
        
        # X6:retained earnings / total assets 
        'x6': lambda obj, i: obj['Retained Earnings'][i] / obj['Total Assets'][i],
        
        # X7:EBIT / total assets 
        'x7': lambda obj, i: obj['EBIT'][i] / obj['Total Assets'][i],
        
        # X8:book value of equity / total liabilities 
        # book value of equity = total assests - total liabilities
        'x8': lambda obj, i: (obj['Total Assets'][i]-obj['Total Liabilities'][i]) / obj['Total Liabilities'][i], 
        
        # X9:sales / total assets 
        'x9': lambda obj, i: obj['Revenues'][i] / obj['Total Assets'][i],
        
        # X10:equity / total assets 
        'x10': lambda obj, i: obj['Total Equity'][i] / obj['Total Assets'][i], 
        
        # X11:(gross profit + extraordinary items + financial expenses) / total assets
        # extroordinary items and financial expenses missing
        'x11': lambda obj, i: '', 
        
        # X12:gross profit / short-term liabilities
        # gross profit = revenue - cogs 
        'x12': lambda obj, i: (obj['Revenues'][i]-obj['COGS'][i]) / obj['Current Liabilities'][i],
       
        # X13:(gross profit + depreciation) / sales 
        # depreciation = operating expenses - sg&A = gross profit - net income - sg&A
        # gross profit + depreciation = 2 * gross profit -net income - sg&A
        'x13': lambda obj, i: (2 * (obj['Revenues'][i]-obj['COGS'][i]) - obj['Net Profit'][i] - obj['SG&A'][i]) / obj['Revenues'][i],
        
        # X14:(gross profit + interest) / total assets 
        # interest missing
        'x14': lambda obj, i:  '', 
        
        # X15:(total liabilities * 365) / (gross profit + depreciation) 
        # gross profit + depreciation = 2 * gross profit -net income - sg&A
        'x15': lambda obj, i: (obj['Total Liabilities'][i] * 365) / (2 * (obj['Revenues'][i]-obj['COGS'][i]) - obj['Net Profit'][i] - obj['SG&A'][i]), 
        
        # X16:(gross profit + depreciation) / total liabilities 
        'x16': lambda obj, i: (2 * (obj['Revenues'][i]-obj['COGS'][i]) - obj['Net Profit'][i] - obj['SG&A'][i]) / obj['Total Liabilities'][i], 
        
        # X17:total assets / total liabilities 
        'x17': lambda obj, i: obj['Total Assets'][i] / obj['Total Liabilities'][i],
        
        # X18:gross profit / total assets
        'x18': lambda obj, i: (obj['Revenues'][i]-obj['COGS'][i]) / obj['Total Assets'][i],
        
        # X19:gross profit / sales 
        'x19': lambda obj, i: (obj['Revenues'][i]-obj['COGS'][i]) / obj['Revenues'][i],
        
         # X20:(inventory * 365) / sales
        # inventory missing
        'x20': lambda obj, i: '', 
        
        'x21': lambda obj, i: '',
        'x22': lambda obj, i: obj['Cash From Operating Activities'][i] / obj['Total Assets'][i], # assume cash from operating activities === cash from operating activities
        'x23': lambda obj, i: obj['Net Profit'][i] / obj['Total Assets'][i],
        'x24': lambda obj, i: '', # gross profit (in 3 years) missing 
        'x25': lambda obj, i: (obj['Total Equity'][i] - obj['Share Capital'][i])  / obj['Total Assets'][i],
        'x26': lambda obj, i: '', # depreciation missing 
        'x27': lambda obj, i: '', # financial expenses missing 
        'x28': lambda obj, i: (obj['Current Assets'][i] - obj['Current Liabilities'][i]) / obj['Net PP&E'][i], # assume net pp&e === fixed assets check 53
        'x29': lambda obj, i: math.log10(obj['Total Assets'][i]),
        'x30': lambda obj, i: (obj['Total Liabilities'][i] - obj['Cash and Cash Equivalents'][i]) / obj['Revenues'][i],
        'x31': lambda obj, i: '', # interest missing
        'x32': lambda obj, i: obj['Current Liabilities'][i] * 365 / obj['COGS'][i], # same as #52
        'x33': lambda obj, i: (obj['COGS'][i] + obj['SG&A'][i]) / obj['Current Liabilities'][i], # assume cogs + sg&a === operating expenses
        'x34': lambda obj, i: (obj['COGS'][i] + obj['SG&A'][i]) / obj['Total Liabilities'][i], # assume cogs + sg&a === operating expenses
        'x35': lambda obj, i: '', # missing net revenue
        'x36': lambda obj, i: obj['Revenues'][i] / obj['Total Assets'][i],
        'x37': lambda obj, i: '', # missing inventories
        'x38': lambda obj, i: '', # constant capital ?? === fixed assets === total noncurrent assets??
        'x39': lambda obj, i: '', # missing net revenue
        'x40': lambda obj, i: '', # missing inventories
        
        'x41': lambda obj, i: obj['Total Liabilities'][i] / (obj['EBIT'][i] * (12.0/365.0)),
        'x42': lambda obj, i: obj['EBIT'][i] / obj['Revenues'][i],
        'x43': lambda obj, i: '', # inventory missing
        'x44': lambda obj, i: obj['Receivables'][i] * 365 / obj['Revenues'][i],
        'x45': lambda obj, i: '', # inventory missing
        'x46': lambda obj, i: '', # inventory missing
        'x47': lambda obj, i: '', # inventory missing
        'x48': lambda obj, i: obj['EBITDA'][i] / obj['Total Assets'][i],
        'x49': lambda obj, i: obj['EBITDA'][i] / obj['Revenues'][i],
        'x50': lambda obj, i: obj['Current Assets'][i] / obj['Total Liabilities'][i],
        'x51': lambda obj, i: obj['Current Liabilities'][i] / obj['Total Assets'][i], # assume short term liability === current liability
        'x52': lambda obj, i: obj['Current Liabilities'][i] * 365 / obj['COGS'][i],
        'x53': lambda obj, i: obj['Total Equity'][i] / obj['Total Noncurrent Assets'][i],
        'x54': lambda obj, i: obj['Net PP&E'][i] / obj['Total Noncurrent Assets'][i],
        'x55': lambda obj, i: obj['Current Assets'][i] - obj['Current Liabilities'][i],
        'x56': lambda obj, i: (obj['Revenues'][i] - obj['COGS'][i]) / obj['Revenues'][i],
        'x57': lambda obj, i: '', # inventory missing
        'x58': lambda obj, i: (obj['Revenues'][i] - obj['Net Profit'][i]) / obj['Revenues'][i],
        'x59': lambda obj, i: obj['Total Noncurrent Liabilities'][i] / obj['Total Equity'][i],
        'x60': lambda obj, i: '', # inventory missing
        'x61': lambda obj, i: obj['Revenues'][i] / obj['Receivables'][i],
        'x62': lambda obj, i: obj['Current Liabilities'][i] * 365 / obj['Revenues'][i],
        'x63': lambda obj, i: obj['Revenues'][i] / obj['Current Liabilities'][i],
        'x64': lambda obj, i: obj['Revenues'][i] / obj['Total Noncurrent Assets'][i],
        
#         Z-Score = 1.2A + 1.4B + 3.3C + 0.6D + 1.0E
#         Where:
#         A = working capital / total assets
#         B = retained earnings / total assets
#         C = earnings before interest and tax / total assets
#         D = market value of equity / total liabilities
#         E = sales / total assets
        'zscore': lambda obj, i: 1.2 * ((obj['Current Assets'][i] - obj['Current Liabilities'][i]) / obj['Total Assets'][i]) \
        + 1.4 * (obj['Retained Earnings'][i] / obj['Total Assets'][i]) \
        + 3.3 * (obj['EBIT'][i] / obj['Total Assets'][i]) \
        + 0.6 * (obj['Share Price'][i] * obj['Common Shares Outstanding'][i] / obj['Total Liabilities'][i]) \
        + (obj['Revenues'][i] / obj['Total Assets'][i]),
        
        # roe: net income / equity
        'roe': lambda obj, i: obj['Net Profit'][i] / obj['Total Equity'][i],
        'equity': lambda obj, i: obj['Total Equity'][i],
        'solvency': lambda obj, i: obj['EBIT'][i] / obj['Total Liabilities'][i],
        'debt_equity':lambda obj, i: obj['Total Liabilities'][i] / obj['Total Equity'][i]
    }
#     res = {ratio: [catchError(obj, i, funcs[ratio]) for i in range(len(obj['Share Price']))] for ratio in funcs}
    res = {ratio: [*filter(lambda item: item['value'] and item['value'] != '', [{
        'date': dates[i],
        'value': catchError(obj, i, funcs[ratio])
    } for i in range(min(len(dates), len(obj['Share Price'])))])] for ratio in funcs}
#     print([key for key in res if len(res[key]) == 0])
    res = {**{'company': obj['company'], 'ticker': obj['ticker'], 'bankrupt': obj['company'] in bankrupt_companies}, **{key: get_first_item_of_year(res[key]) for key in res if len(res[key]) > 0}}
    return res

In [7]:
new_data = [get_features(obj) for obj in data]
new_data[0]

{'bankrupt': False,
 'company': '1 800 FLOWERS COM INC',
 'debt_equity': [{'date': '10/11/11', 'value': 1.0021245910865666},
  {'date': '11/5/12', 'value': 0.663752116916515},
  {'date': '8/2/13', 'value': 0.6734745379744236},
  {'date': '7/2/14', 'value': 0.5599597519134336},
  {'date': '6/2/15', 'value': 1.625391648658562},
  {'date': '5/2/16', 'value': 1.4601248373706712},
  {'date': '10/2/17', 'value': 1.1585393070356524},
  {'date': '9/2/18', 'value': 0.9615379963838884}],
 'equity': [{'date': '10/11/11', 'value': 142145000.0},
  {'date': '11/5/12', 'value': 159430000.0},
  {'date': '8/2/13', 'value': 170391000.0},
  {'date': '7/2/14', 'value': 181872000.0},
  {'date': '6/2/15', 'value': 226287000.0},
  {'date': '5/2/16', 'value': 258255999.99999997},
  {'date': '10/2/17', 'value': 286768000.0},
  {'date': '9/2/18', 'value': 330742000.0}],
 'roe': [{'date': '10/11/11', 'value': 0.07109641563192515},
  {'date': '11/5/12', 'value': 0.09914068870350624},
  {'date': '8/2/13', 'value':

In [8]:
with open('data_with_ratios.json', 'w') as fp:
    json.dump(new_data, fp)

In [9]:
with open('data_with_ratios.pickle', 'wb') as fp:
    pickle.dump(new_data, fp)